<a href="https://colab.research.google.com/github/arjelmilan/MA/blob/master/keras_Hyperparamete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler=StandardScaler()

In [11]:
X=scaler.fit_transform(X)

In [13]:
X.shape

(768, 8)

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [15]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [22]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 [==============================] - 0s 23ms/step - loss: 0.4992 - accuracy: 0.7606 - val_loss: 0.4808 - val_accuracy: 0.7857
Epoch 2/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4910 - accuracy: 0.7655 - val_loss: 0.4744 - val_accuracy: 0.7922
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4841 - accuracy: 0.7687 - val_loss: 0.4701 - val_accuracy: 0.7922
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4788 - accuracy: 0.7687 - val_loss: 0.4665 - val_accuracy: 0.7922
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4743 - accuracy: 0.7736 - val_loss: 0.4642 - val_accuracy: 0.7922
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4704 - accuracy: 0.7752 - val_loss: 0.4619 - val_accuracy: 0.7922
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4671 - accuracy: 0.7834 - val_loss: 0.4618 - val_accuracy: 0.7922
Epoch 8/10
20/20 [=

In [25]:
import keras_tuner as kt


ModuleNotFoundError: No module named 'keras_tuner'

In [26]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [28]:
import keras_tuner as kt

In [35]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [36]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [43]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop', 'adadelta'], 'ordered': False}


In [37]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 08s


In [39]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [40]:
model=tuner.get_best_models(num_models=1)[0]

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5813 - accuracy: 0.7378 - val_loss: 0.5635 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5586 - accuracy: 0.7443 - val_loss: 0.5446 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5397 - accuracy: 0.7622 - val_loss: 0.5277 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5233 - accuracy: 0.7622 - val_loss: 0.5163 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5107 - accuracy: 0.7704 - val_loss: 0.5057 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4998 - accuracy: 0.7736 - val_loss: 0.4974 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4906 - accuracy: 0.7687 - val_loss: 0.4907 - val_accuracy: 0.7857
Epoch 14

In [58]:
def build_model(hp):
  model=Sequential()

  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [59]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir')

In [60]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6103895902633667

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [61]:
tuner.get_best_hyperparameters()[0].values

{'units': 40}

In [63]:
model=tuner.get_best_models(num_models=1)[0]

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                360       
                                                                 
 dense_1 (Dense)             (None, 1)                 41        
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5061 - accuracy: 0.7557 - val_loss: 0.5122 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4934 - accuracy: 0.7622 - val_loss: 0.4990 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4835 - accuracy: 0.7638 - val_loss: 0.4918 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4763 - accuracy: 0.7671 - val_loss: 0.4855 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4709 - accuracy: 0.7671 - val_loss: 0.4803 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4664 - accuracy: 0.7671 - val_loss: 0.4779 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4632 - accuracy: 0.7638 - val_loss: 0.4745 - val_accuracy: 0.7987
Epoch 14

In [68]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

      model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [70]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='num_layers')

In [71]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8311688303947449

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 12s


In [72]:
tuner.get_best_hyperparameters()[0].values


{'num_layers': 6}

In [73]:
model=tuner.get_best_models(num_models=1)[0]


In [74]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))


Epoch 7/100
20/20 [==============================] - 3s 15ms/step - loss: 0.4702 - accuracy: 0.7671 - val_loss: 0.4619 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4498 - accuracy: 0.7736 - val_loss: 0.4542 - val_accuracy: 0.8247
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4417 - accuracy: 0.7932 - val_loss: 0.4755 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4321 - accuracy: 0.7997 - val_loss: 0.4946 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4177 - accuracy: 0.8029 - val_loss: 0.5573 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4175 - accuracy: 0.8078 - val_loss: 0.4752 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4013 - accuracy: 0.8046 - val_loss: 0.5004 - val_accuracy: 0.7922
Epoch 14

In [99]:

def build_model(hp):
  model=Sequential()

  counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

      if counter==0:

          model.add(
              Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
              input_dim=8
                          )
          )
      else:
          model.add(
              Dense(
                  hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                  activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                  input_dim=8
                          )
              )
      counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [100]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='mydir1',project_name='num_1layers')
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))





Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 26s


In [101]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1, 'units0': 56, 'activation0': 'relu', 'optimizer': 'rmsprop'}